# PROOF OF CONCEPT WORKSHEET
## Use this worksheet to work out process and solutions.

## To use twint
pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
### Upgrade from a different repo for MACOS & LINUX? if above doesn't work.
* From https://github.com/twintproject/twint/pull/1272
* pip3 install --upgrade --user git+https://github.com/MarcosFP97/twint.git@origin/master#egg=twint


----------------------------

# Process Template for Getting Data to End Solution:

### FUNCTION 1: CREATE LOOP TO FIND TWEETS & STITCH TOGETHER CSV FILES INTO 1 FILE.
* Search for Keywords ['BTC','ETH', 'DOGE', 'SHIBA', 'Bitcoin', 'Ethereum', 'Shiba Inu'], store tweets with keywords

### FUNCTION 2: Filter out rows that aren't related according to Tweet Text, Hashtags, Cashtags, or Mentions

### FUNCTION 3: RUN SENTIMENT ANALYSIS & GET COMPOUND SCORE FOR SAVED TWEETS

### DATA-PREPROCESSING:
* Determine which Linear Values are Normalized, use Standard Scaler
* Determine which Linear Values are Skewed, use MinMax Scaler
* Scaling Linear Values and Get Dummies (OneHotEncoder) for Class Values

### LSTM RNN Deep Learning

--------------------------------

In [47]:
import twint
import nest_asyncio
nest_asyncio.apply()
import pandas as pd

import re
import string
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from datetime import datetime, timedelta

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\erikl\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Configure
# c = twint.Config()
# c.Limit = 1
# c.Username = "@CNNbrk"

# # Run
# twint.run.Search(c)

In [2]:
c = twint.Config()
c.Lang = "en"
c.Username = "elonmusk"
# c.Hide_output = True
c.Since = '2020-10-06'
c.Until= '2021-10-06'
# c.Search = ['BTC','ETH', 'DOGE', 'SHIBA', 'Bitcoin', 'Ethereum', 'Shiba Inu']  -> Using this breaks Twint.
c.Min_likes = 500
c.Min_replies = 50
c.Min_retweets = 100
c.Store_csv = True
c.Output = "test_1.csv"

# Run
twint.run.Search(c)

1445087513585102853 2021-10-04 13:04:46 -0500 <elonmusk> @Teslarati @KlenderJoey This was *extremely* difficult. Huge props to Tesla engineering, supply chain, production &amp; key suppliers.
1444840184500129797 2021-10-03 20:41:58 -0500 <elonmusk> Floki Frunkpuppy  https://t.co/xAr8T0Jfdf
1444793562424614915 2021-10-03 17:36:42 -0500 <elonmusk> @StianWalgermo @Tesla @WholeMarsBlog Wow!
1444778635425206273 2021-10-03 16:37:24 -0500 <elonmusk> @TheStonkJunky Awesome, effortless ordering is our goal :)
1444647286424936452 2021-10-03 07:55:27 -0500 <elonmusk> @PPathole Tough choice – so many great movies –but Spaceballs for me
1444645326053326848 2021-10-03 07:47:40 -0500 <elonmusk> @vm_one1 Lookin good …
1444644149395959808 2021-10-03 07:43:00 -0500 <elonmusk> Walk this way … no, *this* way …  https://t.co/5DHEOh9Rke
1444642184209309699 2021-10-03 07:35:11 -0500 <elonmusk> @waitbutwhy 🤣
1444575088053997568 2021-10-03 03:08:34 -0500 <elonmusk> @SawyerMerritt I wish I had had the opportuni

In [3]:
print(type(c))

<class 'twint.config.Config'>


In [4]:
c = twint.Config()
c.Lang = "en"
c.Username = "elonmusk"
# c.Hide_output = True
c.Since = '2020-10-06'
c.Until= '2021-09-23'
# c.Search = ['BTC','ETH', 'DOGE', 'SHIBA', 'Bitcoin', 'Ethereum', 'Shiba Inu']
c.Min_likes = 500
c.Min_replies = 50
c.Min_retweets = 100
c.Store_csv = True
c.Output = "test_2.csv"

# Run
twint.run.Search(c)

1440164262761021448 2021-09-20 23:01:31 -0500 <elonmusk> @carsonight Not super surprising, given that internal combustion engine cars literally have “combustion” in the name
1440159176685088774 2021-09-20 22:41:19 -0500 <elonmusk> Just met with the @Inspiration4x crew in Florida to congratulate them in person. Such great people!
1440017820167938054 2021-09-20 13:19:37 -0500 <elonmusk> @ray4tesla Tesla has not yet decided on a fourth Gigafactory location
1439840888537194496 2021-09-20 01:36:33 -0500 <elonmusk> @AustinTeslaClub To be precise, completely &amp; immediately reusable orbital rockets are the fundamental breakthrough needed to make life multiplanetary
1439840278362329094 2021-09-20 01:34:07 -0500 <elonmusk> @teslaownersSV @Tesla Which games did they like most?
1439787653394747395 2021-09-19 22:05:01 -0500 <elonmusk> @teslaownersSV @WholeMarsBlog Tesla insurance calculator will show status in real-time &amp; tell you what actions are needed to be rated “good driver”
14397871292

In [5]:
df_1 = pd.read_csv('test_1.csv')
df_1.head(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1445087513585102853,1445086813631221762,2021-10-04 13:04:46 Central Daylight Time,2021-10-04,13:04:46,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Teslarati', 'name': 'TESLARA...",NaN,NaN,NaN,NaN
1,1444840184500129797,1444840184500129797,2021-10-03 20:41:58 Central Daylight Time,2021-10-03,20:41:58,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
df_2 = pd.read_csv('test_2.csv')
df_2.head(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1440164262761021448,1439990574203826181,2021-09-20 23:01:31 Central Daylight Time,2021-09-20,23:01:31,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'carsonight', 'name': 'Carson...",NaN,NaN,NaN,NaN
1,1440159176685088774,1440159176685088774,2021-09-20 22:41:19 Central Daylight Time,2021-09-20,22:41:19,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
df_1.shape

(124, 36)

In [8]:
df_2.shape

(80, 36)

In [9]:
combined = pd.concat([df_1, df_2], join= 'inner')
display(combined.head(3))
display(combined.shape)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1445087513585102853,1445086813631221762,2021-10-04 13:04:46 Central Daylight Time,2021-10-04,13:04:46,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Teslarati', 'name': 'TESLARA...",NaN,NaN,NaN,NaN
1,1444840184500129797,1444840184500129797,2021-10-03 20:41:58 Central Daylight Time,2021-10-03,20:41:58,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1444793562424614915,1444774412113285120,2021-10-03 17:36:42 Central Daylight Time,2021-10-03,17:36:42,-500,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'StianWalgermo', 'name': 'Wal...",NaN,NaN,NaN,NaN


(204, 36)

In [10]:
combined.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

144

79

In [11]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 0 to 79
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               204 non-null    int64  
 1   conversation_id  204 non-null    int64  
 2   created_at       204 non-null    object 
 3   date             204 non-null    object 
 4   time             204 non-null    object 
 5   timezone         204 non-null    int64  
 6   user_id          204 non-null    int64  
 7   username         204 non-null    object 
 8   name             204 non-null    object 
 9   place            0 non-null      float64
 10  tweet            204 non-null    object 
 11  language         204 non-null    object 
 12  mentions         204 non-null    object 
 13  urls             204 non-null    object 
 14  photos           204 non-null    object 
 15  replies_count    204 non-null    int64  
 16  retweets_count   204 non-null    int64  
 17  likes_count      

In [12]:
focused_df = combined[['id', 'conversation_id', 'created_at', 'date', 'time',
       'tweet', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'reply_to']]
focused_df.head(3)

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1445087513585102853,1445086813631221762,2021-10-04 13:04:46 Central Daylight Time,2021-10-04,13:04:46,@Teslarati @KlenderJoey This was *extremely* d...,[],[],[],1623,906,17970,[],[],https://twitter.com/elonmusk/status/1445087513...,False,NaN,0,NaN,"[{'screen_name': 'Teslarati', 'name': 'TESLARA..."
1,1444840184500129797,1444840184500129797,2021-10-03 20:41:58 Central Daylight Time,2021-10-03,20:41:58,Floki Frunkpuppy https://t.co/xAr8T0Jfdf,[],[],['https://pbs.twimg.com/media/FA0afL1X0AIySyt....,28234,30556,350036,[],[],https://twitter.com/elonmusk/status/1444840184...,False,NaN,1,https://pbs.twimg.com/media/FA0afL1X0AIySyt.jpg,[]
2,1444793562424614915,1444774412113285120,2021-10-03 17:36:42 Central Daylight Time,2021-10-03,17:36:42,@StianWalgermo @Tesla @WholeMarsBlog Wow!,[],[],[],629,334,8617,[],[],https://twitter.com/elonmusk/status/1444793562...,False,NaN,0,NaN,"[{'screen_name': 'StianWalgermo', 'name': 'Wal..."


In [13]:
focused_df.shape

(204, 20)

In [14]:
print(f"Total ID count: {combined['id'].count()}")
print(f"Unique IDs: {combined['id'].nunique()}")
# display(print(combined['conversation_id'].nunique())

Total ID count: 204
Unique IDs: 93


In [15]:
print(f"Total Conversation ID count: {combined['conversation_id'].count()}")
print(f"Unique Conversation IDs: {combined['conversation_id'].nunique()}")

Total Conversation ID count: 204
Unique Conversation IDs: 79


In [36]:
# remove_duplicates = focused_df.drop(columns = ['id', 'conversation_id'])

In [46]:
# duplicates = focused_df[['id', 'conversation_id']].drop_duplicates

In [53]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\erikl\anaconda3\envs\nlpenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [16]:
duplicates_new = focused_df.groupby(['id', 'conversation_id','link'])['tweet']
duplicates_newDF = pd.DataFrame(duplicates_new)
display(duplicates_newDF.head())
display(duplicates_newDF.shape)

,0,1
0,"(1438373866762612738, 1438236070534025218, htt...",39 @inspiration4x @ChrisSembroski @rookisaa...
1,"(1438553356520378374, 1438535180407377923, htt...",38 @JeffBezos @SpaceX Thank you Name: tweet...
2,"(1438558174471233537, 1438503135312429059, htt...",37 @richardbranson @SpaceX @inspiration4x T...
3,"(1438569355328036869, 1438380786533552129, htt...",36 @teslaownersSV Maneuverability is import...
4,"(1438579102240346121, 1438576819117215749, htt...",35 @jack That’s my safe word too Name: twee...


(93, 2)

In [17]:
duplicates_new.first().head()

id                   conversation_id      link                                                   
1438373866762612738  1438236070534025218  https://twitter.com/elonmusk/status/1438373866762612738    @inspiration4x @ChrisSembroski @rookisaacman @...
1438553356520378374  1438535180407377923  https://twitter.com/elonmusk/status/1438553356520378374                         @JeffBezos @SpaceX Thank you
1438558174471233537  1438503135312429059  https://twitter.com/elonmusk/status/1438558174471233537    @richardbranson @SpaceX @inspiration4x Thanks ...
1438569355328036869  1438380786533552129  https://twitter.com/elonmusk/status/1438569355328036869    @teslaownersSV Maneuverability is important fo...
1438579102240346121  1438576819117215749  https://twitter.com/elonmusk/status/1438579102240346121                        @jack That’s my safe word too
Name: tweet, dtype: object

-----------------------------------------

## Removing duplicates from the DataFrame:

In [18]:
focused_df.drop_duplicates(inplace=True)

C:\Users\erikl\anaconda3\envs\nlpenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
focused_df.shape

(167, 20)

In [21]:
# # Displays all twitter info in a column
# for index, row in focused_df.iterrows():
#     print(index, row)

## Filter to show only rows with relevant tweets:

In [22]:
import re
ids = focused_df['mentions'].str.contains('Bitcoin|btc|Ethereum|eth|Doge|Shiba|inspiration', flags = re.IGNORECASE, regex = True, na = False)

In [25]:
hashtag_T = focused_df['hashtags'].str.contains('Bitcoin|btc|Ethereum|eth|Doge|Shiba|inspiration', flags = re.IGNORECASE, regex = True, na = False)

In [26]:
cashtags_T = focused_df['cashtags'].str.contains('Bitcoin|btc|Ethereum|eth|Doge|Shiba|inspiration', flags = re.IGNORECASE, regex = True, na = False)

In [29]:
focused_df[hashtag_T==True].head()

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [32]:
focused_df[cashtags_T==True].head()

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [28]:
focused_df[ids ==True].head()

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
10,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,4142,45086,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
43,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,4142,45093,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
94,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5358,4285,46951,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
1,1440159176685088774,1440159176685088774,2021-09-20 22:41:19 Central Daylight Time,2021-09-20,22:41:19,Just met with the @Inspiration4x crew in Flori...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],5704,4444,88674,[],[],https://twitter.com/elonmusk/status/1440159176...,False,NaN,0,NaN,[]
16,1439366880310046721,1439366880310046721,2021-09-18 18:13:01 Central Daylight Time,2021-09-18,18:13:01,Congratulations @Inspiration4x!!!,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],8232,8659,125527,[],[],https://twitter.com/elonmusk/status/1439366880...,False,NaN,0,NaN,[]


In [24]:
print(ids)

0     False
1     False
2     False
3     False
4     False
      ...  
75    False
76     True
77    False
78    False
79    False
Name: mentions, Length: 167, dtype: bool


## Combined DataFrame with all the filters in place:

In [40]:
# Create a DataFrame that has any of the searchterms in Hashtag, mention, cashtag, or tweet:
subject_tweets = focused_df[(ids | hashtag_T | cashtags_T) == True].reset_index(drop=True)
display(subject_tweets.shape)
subject_tweets.head(10)

(8, 20)

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,4142,45086,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
1,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,4142,45093,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
2,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5358,4285,46951,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[]
3,1440159176685088774,1440159176685088774,2021-09-20 22:41:19 Central Daylight Time,2021-09-20,22:41:19,Just met with the @Inspiration4x crew in Flori...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],5704,4444,88674,[],[],https://twitter.com/elonmusk/status/1440159176...,False,NaN,0,NaN,[]
4,1439366880310046721,1439366880310046721,2021-09-18 18:13:01 Central Daylight Time,2021-09-18,18:13:01,Congratulations @Inspiration4x!!!,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],8232,8659,125527,[],[],https://twitter.com/elonmusk/status/1439366880...,False,NaN,0,NaN,[]
5,1438591328082923521,1438591328082923521,2021-09-16 14:51:15 Central Daylight Time,2021-09-16,14:51:15,Just spoke with @inspiration4x crew. All is well.,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],6685,5600,109451,[],[],https://twitter.com/elonmusk/status/1438591328...,False,NaN,0,NaN,[]
6,1440159176685088774,1440159176685088774,2021-09-20 22:41:19 Central Daylight Time,2021-09-20,22:41:19,Just met with the @Inspiration4x crew in Flori...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],5718,4405,88651,[],[],https://twitter.com/elonmusk/status/1440159176...,False,NaN,0,NaN,[]
7,1439366880310046721,1439366880310046721,2021-09-18 18:13:01 Central Daylight Time,2021-09-18,18:13:01,Congratulations @Inspiration4x!!!,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],8240,8635,125515,[],[],https://twitter.com/elonmusk/status/1439366880...,False,NaN,0,NaN,[]


# Sentiment Analysis:

- Clean the Text: 1. Tokenize the text. 2. Put text in Lower Case.  3. Regex to remove punctuation.  4. Remove StopWords
- Apply Vader Sentiment Analysis, get Compound Sentiment & save to new Sentiment Analysis column
- 

### Clean the Text:

In [45]:
## Combine these two functions into one and create a new column for Sentiment Analysis Compound Score:
# Function to clean text
def clean_text(text):
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    return re_clean


# Function to get compound sentiments
def get_compound_sentiment(content):
    sentiment = analyzer.polarity_scores(content)
    compound = sentiment["compound"]
    return compound
# get_compoundScore = lambda x: get_compound_sentiment(x)

In [43]:
# Test the clean_text(text) function
test_clean = clean_text(subject_tweets.loc[0]['tweet'])
display(type(test_clean))
test_clean

str

'Amazing show about Inspirationx mission  httpstconQuajGiz'

In [48]:
# Test the get compound sentiment:
test_compound = get_compound_sentiment(test_clean)
test_compound

0.5859

In [49]:
# Create single function to build Sentiment Analysis Compound Score column:
def compound_sentiment_columns(text):
    # Function to clean text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    # Function to get compound sentiments
    sentiment = analyzer.polarity_scores(re_clean)
    compound = sentiment["compound"]
    return compound

In [50]:
# Use function to create Sentiment Compound score in DataFrame:
subject_tweets['Compound Sentiment'] = subject_tweets['tweet'].apply(lambda x: compound_sentiment_columns(x))
subject_tweets.head()

,id,conversation_id,created_at,date,time,tweet,mentions,urls,photos,replies_count,...,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to,Compound Sentiment
0,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,...,45086,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[],0.5859
1,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5091,...,45093,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[],0.5859
2,1444424110961041408,1444424110961041408,2021-10-02 17:08:38 Central Daylight Time,2021-10-02,17:08:38,Amazing show about @Inspiration4x mission! ht...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",['https://www.netflix.com/title/81441273'],[],5358,...,46951,[],[],https://twitter.com/elonmusk/status/1444424110...,False,NaN,0,NaN,[],0.5859
3,1440159176685088774,1440159176685088774,2021-09-20 22:41:19 Central Daylight Time,2021-09-20,22:41:19,Just met with the @Inspiration4x crew in Flori...,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],5704,...,88674,[],[],https://twitter.com/elonmusk/status/1440159176...,False,NaN,0,NaN,[],0.8074
4,1439366880310046721,1439366880310046721,2021-09-18 18:13:01 Central Daylight Time,2021-09-18,18:13:01,Congratulations @Inspiration4x!!!,"[{'screen_name': 'inspiration4x', 'name': 'ins...",[],[],8232,...,125527,[],[],https://twitter.com/elonmusk/status/1439366880...,False,NaN,0,NaN,[],0.5994
